In [1]:
import pandas as pd

In [2]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

----

In [193]:
airports_df = pd.read_csv('../data/airport_dataset/reference_w_airports.csv')
sports_df = pd.read_csv('../data/sports_dataset/reference_w_sports.csv')
population_df = pd.read_csv('../data/population_dataset/reference_w_population.csv')
meteorite_df = pd.read_csv('../data/meteorite_dataset/reference_w_meteorite.csv')

In [194]:
ref_df = pd.read_csv('../data/reference_w_loc.csv')

#### Sanity check: make sure all DFs have same number of rows

In [195]:
len(ref_df.index) == len(airports_df.index) == len(sports_df.index) == len(population_df.index) == len(meteorite_df.index)

False

In [196]:
ref_df.shape

(38511, 12)

In [197]:
airports_df.shape

(38511, 20)

In [198]:
sports_df.shape

(38511, 17)

In [199]:
population_df.shape

(38439, 14)

In [200]:
meteorite_df.shape

(38511, 17)

### Merge ref with airports

In [201]:
airports_df.columns

Index([u'id', u'description', u'duration', u'location', u'reported_at',
       u'shape', u'sighted_at', u'geocoded_latitude', u'geocoded_longitude',
       u'city', u'state', u'country', u'closest_LARGE_airport_name',
       u'closest_LARGE_airport_distance', u'closest_MEDIUM_airport_name',
       u'closest_MEDIUM_airport_distance', u'closest_SMALL_airport_name',
       u'closest_SMALL_airport_distance', u'closest_CLOSED_airport_name',
       u'closest_CLOSED_airport_distance'],
      dtype='object')

In [202]:
airports_subset_df = airports_df[['id','geocoded_latitude','geocoded_longitude','closest_LARGE_airport_distance', 'closest_MEDIUM_airport_distance','closest_SMALL_airport_distance', 'closest_LARGE_airport_name', 'closest_MEDIUM_airport_name', 'closest_SMALL_airport_name']]

In [203]:
airports_subset_df.columns.values[1] = 'airport_lat'
airports_subset_df.columns.values[2] = 'airport_lon'

In [204]:
airports_subset_df.head(1)

,id,airport_lat,airport_lon,closest_LARGE_airport_distance,closest_MEDIUM_airport_distance,closest_SMALL_airport_distance,closest_LARGE_airport_name,closest_MEDIUM_airport_name,closest_SMALL_airport_name
0,0,41.661256,-91.529911,12.498008,6.616011,0.683006,The Eastern Iowa Airport,Chippewa Valley Regional Airport,Marion Airport


In [205]:
ref_airports = ref_df.merge(airports_subset_df, on='id')

In [206]:
ref_airports.head()

,id,description,duration,location,reported_at,shape,sighted_at,geocoded_latitude,geocoded_longitude,city,state,country,airport_lat,airport_lon,closest_LARGE_airport_distance,closest_MEDIUM_airport_distance,closest_SMALL_airport_distance,closest_LARGE_airport_name,closest_MEDIUM_airport_name,closest_SMALL_airport_name
0,0,"Man repts. witnessing &quot;flash, followed by...",NaN,"Iowa City, IA",19951009,unknown,19951009,41.661256,-91.529911,Iowa City,Iowa,us,41.661256,-91.529911,12.498008,6.616011,0.683006,The Eastern Iowa Airport,Chippewa Valley Regional Airport,Marion Airport
1,1,"Man on Hwy 43 SW of Milwaukee sees large, bri...",2 min.,"Milwaukee, WI",19951011,unknown,19951010,43.034993,-87.922497,Milwaukee,Wisconsin,us,43.034993,-87.922497,1.801887,1.157715,1.716202,General Mitchell International Airport,Kenosha Regional Airport,Herbert C. Maas Airport
2,2,Telephoned Report:CA woman visiting daughter w...,NaN,"Shelton, WA",19950103,unknown,19950101,47.215094,-123.100707,Shelton,Washington,us,47.215094,-123.100707,43.233065,16.484896,3.339607,McChord Air Force Base,Olympia Regional Airport,Sanderson Field
3,3,Man repts. son&apos;s bizarre sighting of smal...,2 min.,"Columbia, MO",19950510,unknown,19950510,38.951883,-92.333737,Columbia,Missouri,us,38.951883,-92.333737,13.834501,7.889876,3.120229,Bill & Hillary Clinton National Airport/Adams ...,Columbia Regional Airport,Willhite Airport
4,4,Anonymous caller repts. sighting 4 ufo&apos;s ...,NaN,"Seattle, WA",19950614,unknown,19950611,47.603832,-122.330062,Seattle,Washington,us,47.603832,-122.330062,3.343976,11.644950,8.871067,Boeing Field King County International Airport,Snohomish County (Paine Field) Airport,Renton Municipal Airport


### Merge ref_airports with sports

In [207]:
sports_df.columns

Index([u'id', u'description', u'duration', u'location', u'reported_at',
       u'shape', u'sighted_at', u'geocoded_latitude', u'geocoded_longitude',
       u'city', u'state', u'country', u'closest_metro_name',
       u'closest_metro_distance', u'closest_metro_pop', u'closest_metro_m4',
       u'closest_metro_m6'],
      dtype='object')

In [208]:
sports_df.closest_metro_pop = sports_df.closest_metro_pop.str.replace(",","").astype(float)

In [209]:
sports_subset_df = sports_df[['id', 'geocoded_latitude', 'geocoded_longitude', 'closest_metro_distance', 'closest_metro_pop', 'closest_metro_m4', 'closest_metro_m6']]

In [210]:
sports_subset_df.columns.values[1] = 'sports_metro_lat'
sports_subset_df.columns.values[2] = 'sports_metro_lon'

In [211]:
ref_airports_sports = ref_airports.merge(sports_subset_df, on='id')

### Merge ref_airports_sports with population

In [212]:
population_df.columns

Index([u'id', u'description', u'duration', u'location', u'reported_at',
       u'shape', u'sighted_at', u'geocoded_latitude', u'geocoded_longitude',
       u'city', u'state', u'country', u'year', u'population'],
      dtype='object')

In [213]:
pop_df = population_df[['id','population']]

In [214]:
pop_df.columns

Index([u'id', u'population'], dtype='object')

In [215]:
ref_airports_sports_population = ref_airports_sports.merge(pop_df, on='id')

In [216]:
ref_airports_sports_population.shape

(38439, 27)

### Merge ref_airports_sports_population with meteorite

In [217]:
meteorite_df.columns

Index([u'id', u'description', u'duration', u'location', u'reported_at',
       u'shape', u'sighted_at', u'geocoded_latitude', u'geocoded_longitude',
       u'city', u'state', u'country', u'meteor_name', u'distance_to_meteor',
       u'reason', u'metorite_lat', u'metorite_long'],
      dtype='object')

In [218]:
len(meteorite_df['distance_to_meteor'].notnull())

38511

In [219]:
meteorite_df['meteor_sighting'] = meteorite_df['distance_to_meteor'].notnull().astype(int)

In [220]:
meteorite_subset_df = meteorite_df[['id', 'metorite_lat', 'metorite_long','meteor_sighting']]

In [221]:
# meteorite_subset_df.columns.values[1] = 'meteorite_lat'
# meteorite_subset_df.columns.values[2] = 'meteorite_lon'

In [222]:
final_df = ref_airports_sports_population.merge(meteorite_subset_df, on='id')

In [223]:
final_df.shape

(38439, 30)

In [224]:
final_df.sample()

,id,description,duration,location,reported_at,shape,sighted_at,geocoded_latitude,geocoded_longitude,city,...,sports_metro_lat,sports_metro_lon,closest_metro_distance,closest_metro_pop,closest_metro_m4,closest_metro_m6,population,metorite_lat,metorite_long,meteor_sighting
5648,8258,Large object just hovered over the vehicle for...,30 sec,"Ashland, OR",20000804,disk,20000730,42.197249,-122.7154,Ashland,...,42.197249,-122.7154,123.96248,2000000.0,1.0,2.0,3421399,NaN,NaN,0


In [227]:
# final_df.to_csv("ufo_dataset_final.csv" ,index=False)
final_df.to_csv("ufo_dataset_final.tsv" ,index=False, sep="\t")

In [166]:
# final_df.to_csv("ufo_dataset_final_with_lat_lon.csv" ,index=False)

In [226]:
# final_df.to_pickle('../data/pickle_files/ufo_dataset_final.pkl')
# final_df = pd.read_pickle('../data/pickle_files/ufo_dataset_final.pkl')

In [185]:
# final_df.to_pickle('../data/pickle_files/ufo_dataset_final_with_lat_lon.pkl')
# final_df = pd.read_pickle('../data/pickle_files/ufo_dataset_final_with_lat_lon.pkl')

In [186]:
ftrs = ['closest_LARGE_airport_distance', 'closest_MEDIUM_airport_distance', 'closest_SMALL_airport_distance', 'closest_metro_distance', 'closest_metro_pop', 'closest_metro_m4', 'closest_metro_m6', 'population', 'meteor_sighting']

In [187]:
df_only_ftrs = final_df[ftrs]

In [188]:
normalized_final_df=(df_only_ftrs-df_only_ftrs.min())/(df_only_ftrs.max()-df_only_ftrs.min())

In [189]:
normalized_final_df['id'] = final_df['id']
# airport coords
# normalized_final_df['airport_lat'] = final_df['airport_lat']
# normalized_final_df['airport_lon'] = final_df['airport_lon']
# # sports coords
# normalized_final_df['sports_metro_lat'] = final_df['sports_metro_lat']
# normalized_final_df['sports_metro_lon'] = final_df['sports_metro_lon']
# # meteor coords
# normalized_final_df['meteorite_lat'] = final_df['meteorite_lat']
# normalized_final_df['meteorite_lon'] = final_df['meteorite_lon']

In [190]:
# normalized_final_df.to_csv('normalized_dataset_final.csv',index=False)
# normalized_final_df.to_csv('normalized_dataset_final_with_lat_lon.csv',index=False)
normalized_final_df.to_csv('normalized_dataset_final_with_lat_lon.tsv',index=False, sep="\t")

In [38]:
final_df.columns

Index([u'id', u'description', u'duration', u'location', u'reported_at',
       u'shape', u'sighted_at', u'geocoded_latitude', u'geocoded_longitude',
       u'city', u'state', u'country', u'airport_lat', u'airport_lon',
       u'closest_LARGE_airport_distance', u'closest_MEDIUM_airport_distance',
       u'closest_SMALL_airport_distance', u'closest_LARGE_airport_name',
       u'closest_MEDIUM_airport_name', u'closest_SMALL_airport_name',
       u'sports_metro_lat', u'sports_metro_lon', u'closest_metro_distance',
       u'closest_metro_pop', u'closest_metro_m4', u'closest_metro_m6',
       u'population', u'meteorite_lat', u'meteorite_lon', u'meteor_sighting'],
      dtype='object')

In [39]:
final_df['airport_lat'].unique()

array([ 41.6612561 ,  43.0349931 ,  47.2150945 , ...,  43.84956865,
        60.684882  ,  38.89764285])

In [40]:
final_df['airport_lon'].unique()

array([ -91.5299106 ,  -87.922497  , -123.1007066 , ...,  -87.73573141,
       -151.1222665 ,  -77.00614639])

In [45]:
df_lat_lon = final_df.copy()

In [43]:
df_lat_lon = str(final_df['airport_lat']) + str(final_df['airport_lon'])

In [44]:
df_lat_lon

'0        41.661256\n1        43.034993\n2        47.215094\n3        38.951883\n4        47.603832\n5        46.877229\n6        36.166286\n7        39.909834\n8        47.669414\n9        47.479908\n10       39.798976\n11       42.123710\n12       41.423967\n13       33.732052\n14       46.686484\n15       28.542400\n16       36.791225\n17       40.811667\n18       44.100351\n19       26.452025\n20       29.894695\n21       34.092233\n22       44.536547\n23       41.413476\n24       34.540281\n25       47.822294\n26       47.659409\n27       60.543944\n28       39.602061\n29       38.449331\n           ...    \n38409    42.391309\n38410    47.827866\n38411    43.849569\n38412    41.710866\n38413    43.425278\n38414    42.360482\n38415    41.823989\n38416    29.424600\n38417    41.010653\n38418    60.684882\n38419    38.897643\n38420    43.051416\n38421    42.700630\n38422    39.766098\n38423    43.774876\n38424    33.827841\n38425    47.614422\n38426    32.366966\n38427    33.834752\

#### checking if TSV is formmated correctly

In [4]:
pd.read_csv("../ufo_dataset_final.tsv", sep="\t")

,id,description,duration,location,reported_at,shape,sighted_at,geocoded_latitude,geocoded_longitude,city,...,country,closest_LARGE_airport_distance,closest_MEDIUM_airport_distance,closest_SMALL_airport_distance,closest_metro_distance,closest_metro_pop,closest_metro_m4,closest_metro_m6,population,meteor_sighting
0,0,"Man repts. witnessing &quot;flash, followed by...",NaN,"Iowa City, IA",19951009,unknown,19951009,41.661256,-91.529911,Iowa City,...,us,12.498008,6.616011,0.683006,88.981643,2195000.0,2.0,2.0,2926324,0
1,1,"Man on Hwy 43 SW of Milwaukee sees large, bri...",2 min.,"Milwaukee, WI",19951011,unknown,19951010,43.034993,-87.922497,Milwaukee,...,us,1.801887,1.157715,1.716202,0.000000,1415000.0,2.0,2.0,5363675,0
2,2,Telephoned Report:CA woman visiting daughter w...,NaN,"Shelton, WA",19950103,unknown,19950101,47.215094,-123.100707,Shelton,...,us,43.233065,16.484896,3.339607,55.154815,3475000.0,2.0,3.0,5894121,0
3,3,Man repts. son&apos;s bizarre sighting of smal...,2 min.,"Columbia, MO",19950510,unknown,19950510,38.951883,-92.333737,Columbia,...,us,13.834501,7.889876,3.120229,56.543882,2795000.0,4.0,5.0,5595211,0
4,4,Anonymous caller repts. sighting 4 ufo&apos;s ...,NaN,"Seattle, WA",19950614,unknown,19950611,47.603832,-122.330062,Seattle,...,us,3.343976,11.644950,8.871067,0.000000,3475000.0,2.0,3.0,5894121,0
5,6,Female student w/ friend witness huge red ligh...,2 min.,"Fargo, ND",19950419,unknown,19950420,46.877229,-96.789821,Fargo,...,us,45.145376,1.828985,1.182489,36.329172,705000.0,1.0,2.0,642200,0
6,7,"Man repts. bright, multi-colored obj. in NW ni...",NaN,"Las Vegas, NV",19950911,unknown,19950911,36.166286,-115.149225,Las Vegas,...,us,2.536446,3.355640,5.772633,0.000000,2255000.0,1.0,1.0,1998257,0
7,8,Woman reports 2 craft fly over house. Strange...,NaN,"Morton, WA",19950214,unknown,19950115,39.909834,-75.323521,Morton,...,us,5.730217,15.260671,4.085405,11.068613,5595000.0,4.0,5.0,12281054,0
8,9,"Young man w/ 2 co-workers witness tiny, distin...",6 min.,"Redmond, WA",19950915,unknown,19950915,47.669414,-122.123877,Redmond,...,us,13.327121,13.971130,7.683524,14.440464,3475000.0,2.0,3.0,5894121,0
9,10,Man repts. seeing 2x large ships hovering in n...,NaN,"Renton, WA",19950220,unknown,19940801,47.479908,-122.203450,Renton,...,us,7.050918,16.617684,0.993545,9.864172,3475000.0,2.0,3.0,4866692,0
